In [1]:
from transformers import LlamaTokenizer
tokenizer_path = ''
tokenizer_path = '/cognitive_comp/ganruyi/LLamaTokenizer_7B_repro2'
tokenizer = LlamaTokenizer.from_pretrained(tokenizer_path, use_fast=False)

Loading the tokenizer from the `special_tokens_map.json` and the `added_tokens.json` will be removed in `transformers 5`,  it is kept for forward compatibility, but it is recommended to update your `tokenizer_config.json` by uploading it again. You will see the new `added_tokens_decoder` attribute that will store the relevant information.
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.
Special tokens have been added in the vocabulary, make sure the assoc

In [2]:
from transformers import AutoTokenizer, LlamaTokenizer
from transformers import LlamaForCausalLM
import torch
import os
ckpt = '/cognitive_comp/ganruyi/Fengshenbang-LM/fengshen/workspace/ziya_llama2_13b_medical/output/checkpoint-240000'
os.environ["CUDA_VISIBLE_DEVICES"] = '6,7'
model = LlamaForCausalLM.from_pretrained(ckpt, torch_dtype=torch.bfloat16, device_map='auto')
tokenizer = LlamaTokenizer.from_pretrained(ckpt, use_fast=False)
query='<human>:我得了新冠，应该怎样治疗？\n<bot>:'



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/ganruyi/anaconda3/envs/py38/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda113.so
CUDA SETUP: CUDA runtime path found: /home/ganruyi/anaconda3/envs/py38/lib/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 113
CUDA SETUP: Loading binary /home/ganruyi/anaconda3/envs/py38/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda113.so...


/home/ganruyi/anaconda3/envs/py38/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/home/ganruyi/anaconda3/envs/py38/lib/libcudart.so'), PosixPath('/home/ganruyi/anaconda3/envs/py38/lib/libcudart.so.11.0')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:
If you get `CUDA error: invalid device function` errors, the above might be the cause and the solution is to make sure only one ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] in the paths that we search based on your env.
  warn(msg)


[2023-10-02 00:49:24,018] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
# inputs = '<human>:' + query.strip() + '\n<bot>:'
# inputs = '写一个童话故事，名字叫《长颈鹿
# 找工作》\n'
query='<human>:冠心病有什么症状\n<bot>:'
      
input_ids = tokenizer(query, return_tensors="pt").input_ids.cuda()
generate_ids = model.generate(
            input_ids,
            max_new_tokens=512, 
            do_sample = True, 
            top_p = 0.85, 
            temperature = 1.0, 
            repetition_penalty=1., 
            eos_token_id=2,
            bos_token_id=1, 
            pad_token_id=0)
output = tokenizer.batch_decode(generate_ids)[0]
print(output)


spaces_between_special_tokens is deprecated and will be removed in transformers v5. It was adding spaces between `added_tokens`, not special tokens, and does not exist in our fast implementation. Future tokenizers will handle the decoding process on a per-model rule.


<s> <human> :冠心病有什么症状 <bot> : 冠心病是我们常见的一种心脏病，大家都非常的熟悉，这种疾病给人们带来的危害是比较大的，患上这种疾病的患者，很容易会出现一些的症状表现，如果大家发现自己患上这种疾病的话，一定要及时的去医院进行诊治，这样才能够早发现疾病，早治疗疾病。冠心病的患者会出现哪些症状，下面给大家介绍一下。冠心病是非常常见的一种心脏疾病，这种疾病给人们带来的危害是比较大的，患上这种疾病的患者，很容易会出现一些的症状表现，如果大家发现自己患上这种疾病的话，一定要及时的去医院进行诊治，这样才能够早发现疾病，早治疗疾病。第一，冠心病患者会出现胸部疼痛的症状，患者在早期的时候会出现胸痛，或者是一种闷痛的感觉，这种症状都是非常明显的，患者如果有这种症状的话，一定要及时的去医院进行诊断，这样才能够早发现疾病，早治疗疾病。第二，患者还会出现非常强烈的咳嗽，一般情况下，咳嗽都是比较常见的，但是如果患者出现了咳嗽的症状的话，就要注意了，这是非常明显的一种症状表现，患者一旦出现这种症状的话，就要及时的去医院进行诊断，这样才能够避免这种疾病带来的危害。第三，患者还会出现非常严重的疲劳感，在这种症状下，患者在生活中还会出现非常强烈的精神疲劳感，在这种情况下患者也


In [3]:
cnts = [185,148,169,176,106,145,237]
def get_all_acc(accs):
    all_cnt = 0
    score = 0
    for i,cnt in enumerate(cnts):
        all_cnt += cnt
        acc = accs[i]
        score += acc*cnt
    return score/all_cnt

print(get_all_acc([0.3891891681519368,0.4324324032140268,0.6153845789713267,0.4772727001549602,0.594339566571739,0.5586206511296102,0.4810126379319562]))


0.49914233663592017


In [6]:
from fengshen.utils import chinese_char_tokenize
outputs = [chinese_char_tokenize('冠心病有什么症状')]
predicts = [chinese_char_tokenize('冠心病的表现')]
from rouge import Rouge
rouge = Rouge()
scores = rouge.get_scores(predicts, outputs, avg=True) 
print(scores)

{'rouge-1': {'r': 0.375, 'p': 0.5, 'f': 0.4285714236734694}, 'rouge-2': {'r': 0.2857142857142857, 'p': 0.4, 'f': 0.33333332847222225}, 'rouge-l': {'r': 0.375, 'p': 0.5, 'f': 0.4285714236734694}}


In [5]:
from fengshen.utils import chinese_char_tokenize
from nltk.translate.bleu_score import corpus_bleu, modified_precision
def compute_bleu(preditions, labels):
    weights = [(0.5,0.5), (0.333,0.333,0.334)]
    score_nltk = corpus_bleu(labels, preditions, weights=weights)
    # score_nltk = modified_precision(labels, preditions, n=1)
    # score_nltk2 = modified_precision(labels, preditions, n=2)
    return score_nltk
outputs = [[chinese_char_tokenize('冠心病有什么症状')]]
predicts = [chinese_char_tokenize('冠心病的一百种症状')]
s1 = compute_bleu(predicts, outputs)
print(s1)

[0.713124819752481, 0.6417315248668485]


In [4]:
from fengshen.models.baichuan.tokenization_baichuan import BaichuanTokenizer
a = '/cognitive_comp/ganruyi/huggingface_models/baichuan/Baichuan-13B-Base'
tokenizer = BaichuanTokenizer.from_pretrained(a)

In [3]:
# from modeling_chatglm import ChatGLMForConditionalGeneration
import torch
from fengshen.models.chatglm.tokenization_chatglm import ChatGLMTokenizer
from fengshen.models.chatglm.modeling_chatglm import ChatGLMForConditionalGeneration
model_path = '/cognitive_comp/ganruyi/huggingface_models/ChatGLM-Med'
tokenizer = ChatGLMTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = ChatGLMForConditionalGeneration.from_pretrained(model_path,device_map="auto",trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

OSError: Unable to load weights from pytorch checkpoint file for '/cognitive_comp/ganruyi/huggingface_models/ChatGLM-Med/pytorch_model-00001-of-00002.bin' at '/cognitive_comp/ganruyi/huggingface_models/ChatGLM-Med/pytorch_model-00001-of-00002.bin'. If you tried to load a PyTorch model from a TF 2.0 checkpoint, please set from_tf=True.

In [ ]:

tokenizer = AutoTokenizer.from_pretrained(base_model)
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    load_in_8bit=load_8bit,
    torch_dtype=torch.float16,
    device_map="auto",
)
print(f"using lora {lora_weights}")
model = PeftModel.from_pretrained(
    model,
    lora_weights,
    torch_dtype=torch.float16,
)

In [5]:
a = torch.load('/cognitive_comp/ganruyi/huggingface_models/ChatGLM-Med/pytorch_model-00001-of-00002.bin')

AttributeError: module 'torch' has no attribute 'loads'

In [1]:
import transformers
ckpt_path = '/cognitive_comp/ganruyi/Fengshenbang-LM/fengshen/workspace/ziya_llama2_13b_medical/output_datav6/checkpoint-2704'
from transformers import AutoTokenizer

query="[human]:感冒怎么处理？\n[bot]:"
tokenizer = AutoTokenizer.from_pretrained(ckpt_path)
input_ids = tokenizer(query, return_tensors="pt").input_ids.to('cuda:0')
# generate_ids = model.generate(
#             input_ids,
#             max_new_tokens=512, 
#             do_sample = True, 
#             top_p = 0.9)
# output = tokenizer.batch_decode(generate_ids)[0]


NameError: name 'ckpt' is not defined